# Problem 1: Generating Random Boolean Functions

The [Deutsch–Jozsa algorithm](https://en.wikipedia.org/wiki/Deutsch%E2%80%93Jozsa_algorithm) is built around a specific promise: any function it receives is guaranteed to be one of two types. A constant function produces the same output regardless of its input, while a balanced function returns `True` for exactly half of all possible inputs and `False` for the other half.

With four Boolean inputs there are $2^4 = 16$ possible input combinations, so the balanced case requires exactly 8 inputs mapped to `True` and 8 to `False`. The first step is to build a way to generate functions that satisfy this promise.

In [29]:
import random

def random_constant_balanced():
    """
    Returns a randomly chosen Boolean function f: {0,1}^4 -> {0,1}
    that is either constant or balanced. satisfying the promise required
    by the Deutsch-Jozsa algorithm.

    The returned function is one of two options with a 50/50 chance of being selected:

        Constant: returns the same Boolean value for all 16 possible inputs,
                  with the output value (True or False) itself chosen at random.

        Balanced: returns True for exactly 8 of the 16 possible inputs and
                  False for the remaining 8, with the selection of True inputs
                  chosen at random from all possible balanced configurations.
                  
    Reference: Deutsch-Jozsa algorithm
    https://en.wikipedia.org/wiki/Deutsch%E2%80%93Jozsa_algorithm

    Returns
    -------
    callable
        A function f(a, b, c, d) where a, b, c, d are bools, returning a bool.
    """
    # randomly choose: constant (true) or balanced (false)
    if random.choice([True, False]):
        # constant
        
        # randomly pick the constant value , so always return False or always return True
        value = random.choice([False, True])
        
        #ignores inputs and always returns the same value
        def f(a, b, c, d):
            return value  # use the value we picked above 
        
        return f
    
    else:
        # balanced
        
        # Generate 16 input combinations for 4 variables
        # This creates: (F,F,F,F), (F,F,F,T), (F,F,T,F), ..., (T,T,T,T)
        inputs = [(a, b, c, d)
                  for a in (False, True)
                  for b in (False, True)
                  for c in (False, True)
                  for d in (False, True)]
        
        # select exactly 8 of the 16 inputs to return True rest return false
        true_inputs = set(random.sample(inputs, 8))
        
        # returns True only for the selected 8 inputs
        def f(a, b, c, d):
            # Check if current input is one of the "True" inputs
            return (a, b, c, d) in true_inputs  
        
        return f

### Tests

In [30]:
# Helper function to get all outputs of a function f for all combinations of inputs
def get_all_outputs(f):
    return [
        f(a, b, c, d)
        for a in (False, True)
        for b in (False, True)
        for c in (False, True)
        for d in (False, True)
    ]

In [31]:
# Test the function returns a boolean output for all input combinations
f = random_constant_balanced()
for out in get_all_outputs(f):
    assert isinstance(out, bool), f"Expected bool, got {type(out)}"
print("PASSED")

PASSED


In [32]:
# all outputs should constant or balanced
for _ in range(200):
    f = random_constant_balanced()
    outputs = get_all_outputs(f)
    true_count = outputs.count(True)
    is_constant = true_count == 0 or true_count == 16
    is_balanced = true_count == 8
    assert is_constant or is_balanced, f"Got {true_count} Trues — neither constant nor balanced"
print("PASSED")

PASSED


In [33]:
# Test the ratio of constant vs balanced functions is roughly 50/50
constant_count = 0
trials = 1000
for _ in range(trials):
    outputs = get_all_outputs(random_constant_balanced())
    true_count = outputs.count(True)
    if true_count in (0, 16):
        constant_count += 1
ratio = constant_count / trials
assert 0.35 < ratio < 0.65, f"Constant ratio {ratio:.2f} seems off"
print(f"PASSED — constant ratio was {ratio:.2f}")

PASSED — constant ratio was 0.50


In [34]:
seen_true = seen_false = False
for _ in range(500):
    outputs = get_all_outputs(random_constant_balanced())
    if all(outputs):
        seen_true = True
    if not any(outputs):
        seen_false = True
    if seen_true and seen_false:
        break
assert seen_true, "Never saw a constant-True function in 500 runs"
assert seen_false, "Never saw a constant-False function in 500 runs"
print("PASSED")

PASSED


<h1>Problem 2: Classical Testing for Function Type</h1>

In [ ]:
def determine_constant_balanced(f):
    """
    Determines whether a Boolean function f: {0,1}^4 -> {0,1} is constant or balanced
    by evaluating f on inputs until the result can be determined with certainty.

    Stops as soon as the result is certain rather than evaluating all 16 inputs:
        - If two different output values are observed, the function must be balanced
        - If the same output value is seen 9 times, the function must be constant,
          since a balanced function can return any one value at most 8 times

    The maximum number of calls to f required is 9, which occurs when the first
    8 calls all return the same value.

    Reference: Deutsch's algorithm
    https://en.wikipedia.org/wiki/Deutsch%E2%80%93Jozsa_algorithm

    Parameters
    ----------
    f : callable
        A function f(a, b, c, d) where a, b, c, d are bools, returning a bool,
        guaranteed to be either constant or balanced.

    Returns
    -------
    str
        "constant" if f returns the same value for all inputs,
        "balanced" if f returns True for exactly 8 of 16 inputs.
    """
    # generate all 16 input combinations
    inputs = [(a, b, c, d)
              for a in (False, True)
              for b in (False, True)
              for c in (False, True)
              for d in (False, True)]

    first = f(*inputs[0])
    same_count = 1

    for i in range(1, len(inputs)):
        result = f(*inputs[i])

        if result != first:
            # two different outputs seen — must be balanced
            return "balanced"

        same_count += 1

        # a balanced function can have at most 8 of the same value
        # so 9 identical outputs means it must be constant
        if same_count == 9:
            return "constant"
    # if we finish the loop without returning, it means all outputs were the same
    return "constant"

In [36]:
# Test constant functions always return "constant"
for _ in range(200):
    f = random_constant_balanced()
    outputs = get_all_outputs(f)
    if outputs.count(True) in (0, 16):
        assert determine_constant_balanced(f) == "constant", "Expected constant"
print("PASSED")

PASSED


In [37]:
# Test balanced functions always return "balanced"
for _ in range(200):
    f = random_constant_balanced()
    outputs = get_all_outputs(f)
    if outputs.count(True) == 8:
        assert determine_constant_balanced(f) == "balanced", "Expected balanced"
print("PASSED")

PASSED


In [40]:
# Test that the function never uses more than 9 calls
import unittest.mock as mock

for _ in range(500):
    f = random_constant_balanced()
    call_count = 0
    original = f

    def counting_f(a, b, c, d):
        global call_count
        call_count += 1
        return original(a, b, c, d)

    call_count = 0
    determine_constant_balanced(counting_f)
    assert call_count <= 9, f"Used {call_count} calls, expected at most 9"

print("PASSED")

PASSED


<h1>Problem 3: Quantum Oracles</h1>

<h1>Problem 4: Deutsch's Algorithm with Qiskit</h1>

<h1>Problem 5: Scaling to the Deutsch–Jozsa Algorithm</h1>